## Automating Reasoning in Hoare Logic

### Motivation

<br>

<img src="./alr-images/1201.png" width="500" style="display: block; margin: auto;">

<br>

- Manual proofs in Hoare logic are **tedious**.
- We want to **automate** the tedious parts of program verification.
- We assume we are given **loop invariants** (e.g. by humans or static analysis tools).

<hr style="border: none; height: 2px; background-color: white; margin: 24px 0;" />

## Verification Conditions (VCs)

<br>

<img src="./alr-images/1202.png" width="500" style="display: block; margin: auto;">

<br>

### What is a Verification Condition?

- A **first-order logic formula** that, if valid, implies the Hoare triple is valid.
- Workflow:
  1. Analyze the source code and **generate a VC**.
  2. Use a **theorem prover** to check VC validity.
  3. If valid, program correctness is established.

## Two Approaches to VC Generation

<br>

<img src="./alr-images/1203.png" width="500" style="display: block; margin: auto;">

<br>

### 1. Forward (Strongest Postcondition)

- Start from precondition and simulate execution symbolically.
- Generate a **strongest postcondition (SP)**.
- Then check:  
  **SP ⇒ Q**  
  (Postcondition weakening)

**Drawback:** Generates unnecessary info and may include **quantifiers**.

### 2. Backward (Weakest Precondition)

- Start from postcondition and work backwards.
- Compute **weakest precondition (WP)**.
- Then check:  
  **P ⇒ WP**  
  (Precondition strengthening)

✅ This approach:
- Is **goal-directed**.
- Produces **quantifier-free** formulas.

<hr style="border: none; height: 2px; background-color: white; margin: 24px 0;" /> 

## Computing Weakest Preconditions

<br>

<img src="./alr-images/1204.png" width="500" style="display: block; margin: auto;">

<br>

<br>

<img src="./alr-images/1205.png" width="500" style="display: block; margin: auto;">

<br>

### Assignment

- For `x := e`, and postcondition `Q`:  
  $WP(Q, x := e) = Q[x \mapsto e]$  
  (Substitute all occurrences of `x` in `Q` with `e`)

### Sequencing

- For `S1; S2`, and postcondition `Q`:
  1. Compute: $Q' = WP(Q, S2)$
  2. Then: $WP(Q, S1; S2) = WP(Q', S1)$

### Conditional

- For `if C then S1 else S2`, and postcondition `Q`:
  $WP(Q, \text{if } C \text{ then } S1 \text{ else } S2) = (C \Rightarrow WP(Q, S1)) \land (\neg C \Rightarrow WP(Q, S2))$

## Example: Computing WP

<br>

<img src="./alr-images/1206.png" width="500" style="display: block; margin: auto;">

<br>

<br>

<img src="./alr-images/1207.png" width="500" style="display: block; margin: auto;">

<br>

Code:

    x := y + 1
    if x > 0 then z := 1 else z := -1

Postcondition:

    z > 0

Working backwards:
- Then branch ⇒ $1 > 0$ → True
- Else branch ⇒ $-1 > 0$ → False
- So:  
  $(x > 0 \Rightarrow \text{True}) \land (x \leq 0 \Rightarrow \text{False})$  
  simplifies to $x > 0$

Propagate through `x := y + 1`:
- $y + 1 > 0$  
✅ This is the **WP** for the full snippet.

<hr style="border: none; height: 2px; background-color: white; margin: 24px 0;" />

## Handling Loops with Invariants

<br>

<img src="./alr-images/1208.png" width="500" style="display: block; margin: auto;">

<br>

### Challenge:

- We cannot compute exact weakest preconditions for loops.
- Loops may execute an unknown number of times.

### Solution:

<br>

<img src="./alr-images/1209.png" width="500" style="display: block; margin: auto;">

<br>

- Use **Approximate Weakest Preconditions (AWP)**:
  - AWP may be stronger than true WP, but not weaker.
  - As long as $P \Rightarrow AWP$, we're good.

## AWP for Loops

<br>

<img src="./alr-images/12010.png" width="500" style="display: block; margin: auto;">

<br>

### Notation:

    while C do [I] S

Where:
- `I` is the **annotated loop invariant**

<hr style="border: none; height: 2px; background-color: white; margin: 24px 0;" />

### AWP Rule:

- $AWP(Q, \text{while } C \text{ do } [I] S) = I$

Because:
- We must establish that `I` holds before the loop starts.

## What Must Be Verified for Loops?

<br>

<img src="./alr-images/12011.png" width="500" style="display: block; margin: auto;">

<br>

1. $P \Rightarrow I$  
   (Precondition must establish the invariant)

2. $I \land \neg C \Rightarrow Q$  
   (Invariant and loop termination must imply postcondition)

3. $I \land C \Rightarrow WP(I, S)$  
   (Invariant is inductive across loop body)

4. Any nested loops inside `S` must also generate VCs for their invariants.

<hr style="border: none; height: 2px; background-color: white; margin: 24px 0;" />

## Summary: VC Generation Process

<br>

<img src="./alr-images/12012.png" width="500" style="display: block; margin: auto;">

<br>

For any Hoare triple $\{P\} S \{Q\}$ with loops:

- Compute:  
  $P \Rightarrow AWP(Q, S)$  
  using loop invariants as approximations.
  
- Additionally, for each loop:

    while C do [I] S

  Generate:
  - $I \land \neg C \Rightarrow Q$  
  - $I \land C \Rightarrow WP(I, S)$  
  - (Nested loop conditions if needed)

<hr style="border: none; height: 2px; background-color: white; margin: 24px 0;" />

## Soundness and Completeness

<br>

<img src="./alr-images/12013.png" width="500" style="display: block; margin: auto;">

<br>

### Soundness:

- If the VC is valid ⇒ Hoare triple is valid

### Not Complete:

- If the VC is invalid, Hoare triple may still be valid

Why?

1. **Weak invariant**: valid but not strong enough to imply `Q`
2. **Bogus invariant**: not even inductive

<hr style="border: none; height: 2px; background-color: white; margin: 24px 0;" />

## Example: Nested Loops with Invariants

<br>

<img src="./alr-images/12014.png" width="500" style="display: block; margin: auto;">

<br>

**Code:**

    i := 1;
    sum := 0;
    while i ≤ n do [sum ≥ 0]
        j := 0;
        while j ≤ i do [sum ≥ 0 ∧ j ≥ 0]
            sum := sum + j;
            j := j + 1;
        i := i + 1;

**Goal:** prove `sum ≥ 0` at the end.

### VC Generation:

1. **Top-level WP**:
   - Propagates through assignments: `True`
   - Check: `True ⇒ True` ✅

2. **Outer loop**:
   - Show:
     - $sum \geq 0 \land \neg(i \leq n) \Rightarrow sum \geq 0$ ✅
     - $sum \geq 0 \land i \leq n \Rightarrow WP(sum \geq 0, S_1)$  
       (Where $S_1$ is the inner loop + `i := i + 1`)

3. **Inner loop**:
   - Show:
     - $sum \geq 0 \land j \geq 0 \land \neg(j \leq i) \Rightarrow sum \geq 0$ ✅
     - $sum \geq 0 \land j \geq 0 \land j \leq i \Rightarrow WP(sum \geq 0 \land j \geq 0, S_2)$
       - Unroll:
         - After `sum := sum + j`, `j := j + 1`
         - Resulting WP:  
           $sum + j \geq 0 \land j + 1 \geq 0$  
           simplifies to  
           $sum \geq 0 \land j \geq 0$  
           ✅ implied by precondition

### Total Number of Clauses:

- 1 VC for precondition
- 2 VCs for outer loop (invariant strength, inductiveness)
- 2 VCs for inner loop (invariant strength, inductiveness)

✅ **Total = 5 verification conditions**
